In [ ]:
import numpy 
import pandas as pd
import nltk
from nltk.corpus import sentiwordnet as swn
from nltk.corpus import wordnet
import nltk.tag
from nltk.tag import pos_tag
nltk.download('wordnet')
nltk.download('sentiwordnet')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
!pip install py-readability-metrics
from readability import  Readability
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package sentiwordnet to /root/nltk_data...
[nltk_data]   Package sentiwordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [ ]:
import pandas as pd 
df=pd.read_csv('/content/drive/My Drive/Reviews Project/cleaned_review_scottsdale.csv')

In [ ]:
df=df[:50000]

In [ ]:
df

,user_id,business_id,city,review_count,stars,review_id,review_stars,useful,funny,cool,text,date
0,ntlvfPzc8eglqvk92iDIAw,8nP8ghEpT6WFcM6tfqAaGA,Scottsdale,720.0,3.5,ACKoJE2q9WhC3XfMQ_U1Qw,2,1,1,1,In a word: Kitsch.\n\nThis 50's palace of pink...,2008-05-27 22:31:01
1,ntlvfPzc8eglqvk92iDIAw,YjAo1DBb0YuPE_RWmhNyrA,Scottsdale,540.0,4.0,eocDUJxy15ej_-rxyBRVXg,4,2,0,1,One of the most impressive things about The Gr...,2008-05-27 15:43:26
2,ntlvfPzc8eglqvk92iDIAw,YjAo1DBb0YuPE_RWmhNyrA,Scottsdale,540.0,4.0,8GJonsHhPpoQZgpGRoOO1w,4,0,0,2,"After a nearly eight year hiatus, I returned t...",2015-12-08 13:44:41
3,61JaBatv-BJkzYol-ejEAg,7T2Dx26gYKPMFmAWwHgnOA,Scottsdale,463.0,4.0,GGiLxmnWUeoV10Kc11Bj-w,5,1,1,1,"This place is awesome. It's pretty unassuming,...",2010-09-20 05:58:40
4,-28RqJIrddOj9hhMBnzaFA,hgo0jDweUcHMHqNDnQQm9A,Scottsdale,261.0,3.5,K3ZUmU46-dBjuGUVkgGoNQ,3,1,0,0,"Ate here recently for lunch, the food was grea...",2012-07-15 22:47:21
...,...,...,...,...,...,...,...,...,...,...,...,...
49995,5IHZFbqYDXGJ00QlyUMPUQ,NAPuzJcanOp91dE4GXPOow,Scottsdale,923.0,4.0,pn2o8u_WtwxhVfYW5JsOaA,1,1,1,0,Absolutely horrible service & food. We gave th...,2015-03-26 23:16:46
49996,5IHZFbqYDXGJ00QlyUMPUQ,Blfn30Y8tvYq903oLiIf3g,Scottsdale,464.0,4.0,aZARU1YRGbH9TVFNGI2z8g,2,0,0,0,Lifetime members do not get as much as they pr...,2015-06-28 14:14:43
49997,5IHZFbqYDXGJ00QlyUMPUQ,h6Y1-KfN2XBkcrV85CS8yg,Scottsdale,805.0,3.0,5qP8Ym3Ob8bdGW-6JuOS4w,5,1,0,0,My husband and I have always enjoyed our time ...,2014-06-09 01:04:41
49998,5IHZFbqYDXGJ00QlyUMPUQ,yqYtY3-Po4OVPafA9Z-Xyw,Scottsdale,752.0,4.5,mqxPrrtiq0O-IXirGoP04A,5,0,0,0,Awesome and big happy hour! Fantastic edamame ...,2019-02-20 01:33:16


In [ ]:
df1=df[['review_id']]
df1

,review_id
0,ACKoJE2q9WhC3XfMQ_U1Qw
1,eocDUJxy15ej_-rxyBRVXg
2,8GJonsHhPpoQZgpGRoOO1w
3,GGiLxmnWUeoV10Kc11Bj-w
4,K3ZUmU46-dBjuGUVkgGoNQ
...,...
49995,pn2o8u_WtwxhVfYW5JsOaA
49996,aZARU1YRGbH9TVFNGI2z8g
49997,5qP8Ym3Ob8bdGW-6JuOS4w
49998,mqxPrrtiq0O-IXirGoP04A


In [ ]:
def LCMS(text):
  text=str(text)
  text=text.lower()
  sentence = nltk.sent_tokenize(text)
  l=[]
  for sent in sentence:
    l.append(pos_tag(nltk.word_tokenize(sent)))
  adj=[]
  v=[]
  sv=[]
  sav=[]
  dav=[]
  iav=[]
  svl=['like','dislike','love','hate','hear','imagine','impress','smell','see','seem','sound','stay','suppose','taste','think','understand','wish','feel','lack','look','mind','know','need','believe','owe','deny','prefer','have','doubt','resemble','remember','appear','satisfy','possess','please','want','concern','weigh','grow','fit','exist','include','involve','belong','consist','contain','depend','own','agree','disagree','mean','deserve','release','measure','recognise','remain','turn','matter','is','am','are','was','be','being','were','been']
  for i in l:
    for j in i:
      wor=j[0]
      cate=j[1]
      if cate.startswith('J'):
        adj.append(wor)
      elif cate.startswith('V'):
        v.append(wor)
        if wor in svl:
          sv.append(wor)
        else:
          syn = wordnet.synsets(wor)
          c=0
          score=0
          for k in syn:
            if (k.name().split(".")[1])=='v' and swn.senti_synset(k.name()).obj_score()!=1:
              g=swn.senti_synset(k.name())
              score=score+g.obj_score()
              c=c+1
            if c==3:
              break
          if c!=0:
            avg=score/c
            if avg>=0.6:
              sav.append(wor)
            elif avg<0.6 and avg>=0.2:
              iav.append(wor)
            else:
              dav.append(wor)
  ladj=len(adj)
  lsv=len(sv)
  lsav=len(sav)
  liav=len(iav)
  ldav=len(dav)
  if (ladj+lsv+lsav+liav+ldav)!=0:
    LCM_score= (4*ladj + 3*lsv + 2*lsav + 1*liav + 1*ldav)/(ladj+lsv+lsav+liav+ldav)
  else:
    LCM_score=0    
  return LCM_score

In [ ]:
import numpy as np
df1=df1.join(pd.DataFrame(np.vectorize(LCMS)(df['text']),columns=['ABSTRACTION_SCORE']))

In [ ]:
df1['review_extemity']=df['review_stars']-df['stars']

In [ ]:
df1

,review_id
0,ACKoJE2q9WhC3XfMQ_U1Qw
1,eocDUJxy15ej_-rxyBRVXg
2,8GJonsHhPpoQZgpGRoOO1w
3,GGiLxmnWUeoV10Kc11Bj-w
4,K3ZUmU46-dBjuGUVkgGoNQ
...,...
49995,pn2o8u_WtwxhVfYW5JsOaA
49996,aZARU1YRGbH9TVFNGI2z8g
49997,5qP8Ym3Ob8bdGW-6JuOS4w
49998,mqxPrrtiq0O-IXirGoP04A


In [ ]:
def readability(text):
  r=Readability(text)
  try:
    return r.flesch_kincaid().score
  except:
    return 0

In [ ]:
df1=df1.join(pd.DataFrame(np.vectorize(readability)(df['text']),columns=['Readability']))

In [ ]:
df1

,review_id
0,ACKoJE2q9WhC3XfMQ_U1Qw
1,eocDUJxy15ej_-rxyBRVXg
2,8GJonsHhPpoQZgpGRoOO1w
3,GGiLxmnWUeoV10Kc11Bj-w
4,K3ZUmU46-dBjuGUVkgGoNQ
...,...
49995,pn2o8u_WtwxhVfYW5JsOaA
49996,aZARU1YRGbH9TVFNGI2z8g
49997,5qP8Ym3Ob8bdGW-6JuOS4w
49998,mqxPrrtiq0O-IXirGoP04A


In [ ]:
a=pd.DataFrame()
df1['usefulness'] = df['useful']/(df['useful'] + df['funny'] +  df['cool'])

In [ ]:
df2=df1[df1['usefulness'].notnull()]

In [ ]:
df3=df2[df2['Readability']!=0]

In [ ]:
df3=df3.reset_index(drop=True)

In [ ]:
df3

,review_id,ABSTRACTION_SCORE,review_extemity,Readability,usefulness,labels
0,ACKoJE2q9WhC3XfMQ_U1Qw,3.130435,-1.5,9.720717,0.333333,0
1,eocDUJxy15ej_-rxyBRVXg,3.130435,0.0,13.110451,0.666667,1
2,8GJonsHhPpoQZgpGRoOO1w,3.000000,0.0,10.806941,0.000000,0
3,FIyNefQaz9vuvCkJHAOACQ,2.909091,1.0,7.930805,1.000000,1
4,KJwEoh5i79tECOLDMcs1Yg,3.022727,1.0,5.457822,0.333333,0
...,...,...,...,...,...,...
9598,lxFQ-DrE6sPxJaf1FmbCLQ,2.628205,-2.0,4.452438,0.333333,0
9599,DIJNEDWG4c_XnpXxDvcpmg,2.555556,-2.5,8.955959,1.000000,1
9600,Hstgb4u0mDaZ5mkHBkWYxg,2.970588,0.5,5.168512,1.000000,1
9601,pn2o8u_WtwxhVfYW5JsOaA,2.702128,-3.0,4.528440,0.500000,0


In [ ]:
df3['labels']=df3['usefulness'].apply(lambda  x : 1 if x>=0.6 else 0)

In [ ]:
df3

,review_id,ABSTRACTION_SCORE,review_extemity,Readability,usefulness,labels
0,ACKoJE2q9WhC3XfMQ_U1Qw,3.130435,-1.5,9.720717,0.333333,0
1,eocDUJxy15ej_-rxyBRVXg,3.130435,0.0,13.110451,0.666667,1
2,8GJonsHhPpoQZgpGRoOO1w,3.000000,0.0,10.806941,0.000000,0
3,FIyNefQaz9vuvCkJHAOACQ,2.909091,1.0,7.930805,1.000000,1
4,KJwEoh5i79tECOLDMcs1Yg,3.022727,1.0,5.457822,0.333333,0
...,...,...,...,...,...,...
9598,lxFQ-DrE6sPxJaf1FmbCLQ,2.628205,-2.0,4.452438,0.333333,0
9599,DIJNEDWG4c_XnpXxDvcpmg,2.555556,-2.5,8.955959,1.000000,1
9600,Hstgb4u0mDaZ5mkHBkWYxg,2.970588,0.5,5.168512,1.000000,1
9601,pn2o8u_WtwxhVfYW5JsOaA,2.702128,-3.0,4.528440,0.500000,0


In [ ]:
X=df3.values[:,1:5]
Y=df3.values[:,-1]
Y=Y.astype('int')

In [ ]:
X

array([[3.130434782608696, -1.5, 9.720716981132075],
       [3.130434782608696, 0.0, 13.110450576318552],
       [3.0, 0.0, 10.806941176470591],
       ...,
       [2.9705882352941178, 0.5, 5.168512396694215],
       [2.702127659574468, -3.0, 4.5284404924760615],
       [2.9285714285714284, 2.0, 4.548461538461542]], dtype=object)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, Y,test_size=0.25, random_state=0)

In [ ]:
clf = SVC(kernel='rbf')

In [ ]:
clf.fit(X_train, y_train)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [ ]:
y_pred = clf.predict(X_test)

In [3]:
confusion_matrix(y_test, y_pred)

[[930 221]
 [419 831]]


In [4]:
clf.score(X_test, y_test)

0.7334443982
